---
title: "Data Analysis using Regression Models"
author: "Just1a2Noob"
format:
  html:
    theme: zephyr
    toc: true
    code-fold: true
    code-summary: "Show the code"
jupyter: python3
---

# Introduction

In this analysis we are going to use regression models used in Scikit-Learn to predict whether an individual has theyre loan approved and what important features that associate with the prediction.

The metadata description of as follows:

| Column | Description | Type |
| --------------- | --------------- | --------------- |
|person_age |Age of the person | Float |
|person_gender	| Gender of the person | Categorical|
|person_education	| Highest education level | Categorical|
|person_income	|  Annual income |Float|
|person_emp_exp	| Years of employment experience | Integer|
|person_home_ownership	| Home ownership status (e.g., rent, own, mortgage)| Categorical|
|loan_amnt	| Loan amount requested | Float|
|loan_intent | Purpose of the loan |Categorical|
|loan_int_rate	| Loan interest rat |	Float|
|loan_percent_income | Loan amount as a percentage of annual income | Float|
|cb_person_cred_hist_length | Length of credit history in years | Float|
|credit_score | Credit score of the person | Integer|
|previous_loan_defaults_on_file | Indicator of previous loan defaults | Categorical|
|loan_status (target variable) | Loan approval status: 1 = approved; 0 = rejected | Integer|

## Setting Up Our Environment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("loan_data.csv")
df.head()

In [ ]:
import matplotlib.pyplot as plt

df.hist(bins=100, figsize=(20, 15))
plt.show()

The graphs above gives us graph of all numerical columns in terms of frequency. From a glance we can see that the column's


In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# This if for splitting the data
train_set, test_set = train_test_split(df, test_size=0.2)

# stratified sampling on the target
stratified_shuffle = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in stratified_shuffle.split(df, df["loan_status"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

loan_status = strat_train_set.copy()

## Exploratory Data Analysis 

In [ ]:
from pandas.plotting import scatter_matrix

attributes = [
    "person_age",
    "person_income",
    "loan_percent_income",
    "credit_score",
    "person_emp_exp",
    "loan_amnt",
]
scatter_matrix(loan_status[attributes], figsize=(12, 8))

The graph above is a correlation matrix shown in terms of graphs. The columns used are age, income, loan percent income, credit score, employment experience, and loan amount. From the graph we can see that multiple graphs are scattered making it hard to find any meaningful patterns just from graphs alone. But we can see outliers and anomalies from these graphs.

We can see an anomaly in the data in the age group there are some people who are more than 80 years old.

In [ ]:
df.loc[df['person_age'] >= 80]

Also, we see an outlier in the person employed experience, some reaching above 100.

In [ ]:
df.loc[df['person_emp_exp'] >= 100]

And finally individuals who have an abnormally large income and abnormally low income we are going to ignore that and accept them because they are within the realm of possibilities but at the extreme level.

## Preparing for Data For Machine Learning 
I am gonna apply both OrdinalEncoder, OneHotEncoder, and StandardScaler to our inputs data set.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler


class Convert_Encoder(BaseEstimator, TransformerMixin):
    # Converts previous_loan_defaults_on_file column to binary
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        enc = OrdinalEncoder(categories=[["No", "Yes"]])
        X["previous_loan_defaults_on_file"] = enc.fit_transform(
            X[["previous_loan_defaults_on_file"]]
        ).astype(int)

        return X


class Convert_Categorical(BaseEstimator, TransformerMixin):
    # Apply OneHotEncoder to every categorical column
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        categorical_columns = [
            "person_gender",
            "person_education",
            "person_home_ownership",
            "loan_intent",
        ]
        ohe = OneHotEncoder()

        for col in categorical_columns:
            unique_values = X[col].unique()
            column_names = [str(value) for value in unique_values]

            matrix = ohe.fit_transform(X[[col]]).toarray()

            for i in range(len(matrix.T)):
                X[str(col) + "_" + column_names[i]] = matrix.T[i]

            X = X.drop([col], axis=1)

        return X


class CustomStandardScaler(BaseEstimator, TransformerMixin):
    # Applies StandardScaler and keeps the column names
    def __init__(self):
        # Initialize the standard scaler
        self.scaler = StandardScaler()
        # Store column names for later use
        self.columns = None

    def fit(self, X, y=None):
        # Store column names before scaling
        self.columns = X.columns
        # Fit the scaler
        self.scaler.fit(X)
        return self

    def transform(self, X):
        # Transform the data
        scaled_data = self.scaler.transform(X)
        # Convert back to DataFrame with original column names
        return pd.DataFrame(scaled_data, columns=self.columns, index=X.index)


X_inputs = strat_test_set.drop("loan_status", axis=1)
y_labels = strat_test_set["loan_status"].copy()

pipe = Pipeline(
    [
        ("Converting yes/no column", Convert_Encoder()),
        ("Converts categorical columns", Convert_Categorical()),
        ("Std scaler", CustomStandardScaler()),
    ]
)

X_transformed = pipe.fit_transform(X_inputs)

X_transformed.head()

## Regression Models
Our first model is linear regression. Below is the results of using Linear Regression model:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_transformed, y_labels)
linear_pred = lin_reg.predict(X_transformed)
linear_rmse = mean_squared_error(y_labels, linear_pred)
print(f"The RMSE of Linear regression is: {linear_rmse}")

And below is Decision Tree regression:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_transformed, y_labels)
tree_pred = tree_reg.predict(X_transformed)
tree_rmse = mean_squared_error(y_labels, tree_pred)
print(f"The RMSE of DecisionTree regression is: {tree_rmse}")

We can instantly notice that the Decision Tree has an abnormal error rate of 0. This means it is overfitting that data to solve this we are going to use cross validation. For that we are going to use Scikit-Learn's K-fold cross validation feature.

In [ ]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(
    lin_reg, X_transformed, y_labels, scoring="neg_mean_squared_error", cv=10
)
tree_scores = cross_val_score(
  tree_reg, X_transformed, y_labels, scoring="neg_mean_squared_error", cv=10
)


def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())


linear_rmse_scores = np.sqrt(-lin_scores)
tree_rmse_scores = np.sqrt(-tree_scores)

print("------LinearRegression-------")
display_scores(linear_rmse_scores)
print("\n")
print("------DecisionTreeRegressor-------")
display_scores(tree_rmse_scores)

### Ensemble Models
We are going to use simple ensemble models with the purpose of seeing whether using ensemble methods are better compared to linear regression and decision tree. 

The ensemble method we are going to use is Random Forests which works by training many Decision Trees on random subsets of the features, then averaging out their predictions.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(X_transformed, y_labels)

forest_pred = forest_reg.predict(X_transformed)
forest_mse = mean_squared_error(y_labels, forest_pred)
forest_rsme = np.sqrt(forest_mse)
print(f"This is the RMSE of Forest Regression: {forest_rsme}")

In [ ]:
forest_scores = cross_val_score(forest_reg, X_transformed, y_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

From the looks of it Random Forests looks very promising. However, not that the score on the training set is still much lower than on the validation sets, meaning that the model is still overfitting the training set. We can fine-tune Forest Regressor using the `RandomizedSearchCV` class from Scikit-Learn which searches a range of parameters and finds the best parameters.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

parameter_distribs = {
  'n_estimators': randint(low=1, high=200),
  'max_features': randint(low=1, high=20)
}

forest_random_search = RandomizedSearchCV(
  forest_reg,
  param_distributions=parameter_distribs, 
  n_iter=10,
  cv=5, 
  scoring="neg_mean_squared_error",
)
forest_random_search.fit(X_transformed, y_labels)

cvres = forest_random_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
  print(np.sqrt(-mean_score), params)

From here we can see the best parameters of Random Forest model, we can also see each feature.

In [ ]:
importances = forest_random_search.best_estimator_.feature_importances_

feature_importance_df = pd.DataFrame({
  'feature': X_transformed.columns,
  'importance': importances,
})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

feature_importance_df

Although this already provides a clear image of which features are of importance, we don't know if they are negatively or positively correlated with the target. To provide a clear correlation of its features to the target we show SHAP values below:

In [ ]:
import matplotlib.pyplot as plt
import shap

best_model = forest_random_search.best_estimator_
explainer = shap.TreeExplainer(best_model)

shap_values = explainer.shap_values(X_transformed)

# Create a function to analyze and visualize SHAP values
def analyze_shap_values(shap_values, X_df, feature_names):
    # 1. Global Feature Impact Analysis
    print("\nGlobal Feature Impact Analysis:")
    print("=" * 80)
    
    # Calculate mean absolute SHAP values for each feature
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Mean |SHAP|': mean_abs_shap
    })
    feature_importance = feature_importance.sort_values('Mean |SHAP|', ascending=False)
    
    print("\nFeature Importance Rankings:")
    print(feature_importance.to_string(index=False))
    
    # 2. Create Summary Plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_df, plot_type="bar", show=False)
    plt.title("Global Feature Impact (SHAP Values)")
    plt.tight_layout()
    plt.show()
    
    # 3. Create SHAP Summary Violin Plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_df, show=False)
    plt.title("Feature Impact Distribution")
    plt.tight_layout()
    plt.show()
    
    # 4. Analyze Individual Feature Effects
    print("\nDetailed Feature Impact Analysis:")
    print("=" * 80)
    
    for feature in feature_names:
        shap_values_feature = shap_values[:, feature_names.index(feature)]
        mean_shap = np.mean(shap_values_feature)
        abs_mean_shap = np.mean(np.abs(shap_values_feature))
        
        print(f"\nFeature: {feature}")
        print(f"Average SHAP Impact: {mean_shap:.4f}")
        print(f"Average Absolute SHAP Impact: {abs_mean_shap:.4f}")
        print(f"Direction: {'Positive' if mean_shap > 0 else 'Negative'} overall impact")

# Perform the analysis
analyze_shap_values(shap_values, X_transformed, list(X_transformed.columns))

The bar graph is showing us the absolute SHAP values of each feature on how they effect the model. The second graph is showing us when a feature is decreases does it's SHAP value also decrease, for example for the feature `previous_loan_defaults_on_file` shows that as it decrease the SHAP value increase.

::: {.notes}
Although SHAP values might seem to be useful there are 2 limitations that must one be aware of:
1. SHAP values are calculated with the assumption that each feature is **independent**.
2. SHAP is not a measure of "how important a given feature is in the real world", it is simply "how important a given feature is to the model".
:::

## Evaluating Random Forest Model
After cleaning and fine-tuning our Random Forest we now evaluate models using the test set prepared beforehand.

In [ ]:
final_model = forest_random_search.best_estimator_

X_test = strat_test_set.drop("loan_status", axis=1)
Y_test = strat_test_set["loan_status"].copy()

X_test_prepared = pipe.fit_transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

final_rmse

Here we can see the predicted RMSE of the Random Forest with the best parameters we had found. Additionally, we can compute the 95% confidence interval for the test RMSE:

In [ ]:
from scipy import stats

confidence = 0.95

squared_errors = (final_predictions - Y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors),
                         )
        )